In [42]:
# -*- coding: utf-8 -*-
import errno
import urllib.request
import urllib.error
import urllib.parse
import json
import math
from bs4 import BeautifulSoup
import time
import secret
from collections import defaultdict


# 네이버 API
client_id = secret.naver_id
client_secret = secret.naver_secret

def get_blog_count(query, display):
    # OO 데이트로 기본 검색
    encode_query = urllib.parse.quote(query + '데이트')
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + encode_query
    request = urllib.request.Request(search_url)

    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if (response_code == 200):
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode("utf-8"))

        print("Last blog build date : " + str(response_body_dict["lastBuildDate"]))

        if response_body_dict["total"] == 0:
            blog_count = 0
        else:
            blog_total = math.ceil(response_body_dict["total"] / int(display))

            if blog_total >= 1000:
                blog_count = 1000
            else:
                blog_count = blog_total

            print("Blog total : " + str(blog_total))
            print("Blog count : " + str(blog_count))

    return blog_count


def get_blog_post(query, display, start_index):
    # OO 데이트로 기본 검색
    encode_query = urllib.parse.quote(query + ' 데이트')
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + encode_query + \
                 "&start=" + str(start_index) + "&display=" + str(display)
    request = urllib.request.Request(search_url)

    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    all_contents = defaultdict(list)

    if (response_code == 200):
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode("utf-8"))
        print(response_body_dict)
        contents = response_body_dict["items"]
        all_contents["blog"] = contents
        all_place = list()

        for item_index in range(0, len(contents)):
            link = contents[item_index]["link"]
            ori_link = link.replace("?Redirect=Log&logNo=", "/")
            post_code = urllib.request.urlopen(ori_link).read()
            post_soup = BeautifulSoup(post_code, "lxml")

            for mainFrame in post_soup.select("iframe#mainFrame"):
                blog_post_url = "http://blog.naver.com" + mainFrame.get("src")
                blog_post_code = urllib.request.urlopen(blog_post_url).read()
                blog_post_soup = BeautifulSoup(blog_post_code, "lxml")

                try:
                    map_soup = blog_post_soup.find("div", attrs={"class" : "se-module se-module-map-text"})
                    place_text = map_soup.find("a")["data-linkdata"]
                    place_dict = json.loads(place_text)
                    all_place.append(place_dict)

                except:
                    pass
            item_index += 1
            time.sleep(0.1)

        all_contents["place"] = all_place

    return all_contents

test_contents = get_blog_post("용답", 100, 1)
test_contents

{'lastBuildDate': 'Tue, 19 Apr 2022 18:56:23 +0900', 'total': 472, 'start': 1, 'display': 100, 'items': [{'title': '답십리 해장국 맛집 장성식당과 <b>용답</b>카페 부루커피', 'link': 'https://blog.naver.com/clf123?Redirect=Log&logNo=222525733111', 'description': '그리고 용답역 바로 앞에 있는 청계천 앞에 벤치에서 천변 즐기며 부루커피 진짜 맛나게 먹었어요 !! 딱 답십리 데이트코스 - <b>용답데이트</b>코스로 좋은듯합니다! 서울여행와서 묵은 청량리호텔인 더 디자이너스... ', 'bloggername': '내 속엔 내가 너무도 많아', 'bloggerlink': 'https://blog.naver.com/clf123', 'postdate': '20211004'}, {'title': '서울, 매화 여행은 <b>용답</b>역 청계천 매화거리로', 'link': 'https://blog.naver.com/yo_onji?Redirect=Log&logNo=222686137521', 'description': '걷다보면 보이는 홍매화 ♥️ 대나무와 함께 피어있는데 색이 참 예뻤어요 ! 대나무도 푸릇푸릇 넘 예뻤구용 쉴수 있게 벤치도 있었어용! <b>데이트</b>나 산책으로 <b>용답</b>역 매화거리, 청계천 매화거리 추천드려요', 'bloggername': 'MY world', 'bloggerlink': 'https://blog.naver.com/yo_onji', 'postdate': '20220329'}, {'title': '서울 매화 명소 : <b>용답</b>역 청계천 하동매실거리 (ft. 개화 정도... ', 'link': 'https://blog.naver.com/bluesspace?Redirect=Log&logNo=222678216282', 'description': '서울 매화 명소: <b

defaultdict(list, {})

In [33]:
from time import time, localtime, strftime

tm = localtime(time())
strftime("%Y-%m-%d", tm)

'2022-04-19'

In [37]:
import json
import os
from time import time, localtime, strftime

tm = localtime(time())
strftime("%Y-%m-%d", tm)

path = "./date_data/"+ strftime("%Y-%m-%d", tm)

try:
    if not os.path.isdir(path):
        os.mkdir(path)
except OSError as e:
    if e.errno != e.errno.EEXIST:
        print("Failed to create")
        raise

with open(path + "/test2.json", "w") as f:
    json.dump(test_contents, f, indent=4, ensure_ascii=False)

In [ ]:
def save_parquet(query, display, start_index):
    data = get_blog_post(query, display, start_index)

    from pyspark.sql import SparkSession

    MAX_MEMORY = "5g"
    spark = SparkSession.builder.appName("today_dating") \
        .config("spark.executor.memory", MAX_MEMORY) \
        .config("spark.driver.memory", MAX_MEMORY) \
        .getOrCreate()

    df = spark.createDataFrame(data, schema=None)
    df.write.json("./")

    print("수집완료")

if __name__ == '__main__':
    query = ["용답"]
    display = 100
    start = 1
    for q in query:
        blog_count = get_blog_count(q, display)
        for start_index in range(start, blog_count + 1, display):
            print(start_index)
            save_parquet(q, display, start_index)
            print()

In [28]:
import boto3
import secret
import os
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

MAX_MEMORY = "5g"
spark = SparkSession.builder.appName("today_dating") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

s3r = boto3.resource("s3", aws_access_key_id=secret.aws_access_id, aws_secret_access_key=secret.aws_access_secret)
bucket = s3r.Bucket("today-dating")

path = "date-blog/test.json"
file = bucket.Object(path).get()

data = file["Body"].read()

# df = spark.read.json(data)
# df.show()

In [30]:
import json

json.loads(data)

{'blog': [{'title': '답십리 해장국 맛집 장성식당과 <b>용답</b>카페 부루커피',
   'link': 'https://blog.naver.com/clf123?Redirect=Log&logNo=222525733111',
   'description': '그리고 용답역 바로 앞에 있는 청계천 앞에 벤치에서 천변 즐기며 부루커피 진짜 맛나게 먹었어요 !! 딱 답십리 데이트코스 - <b>용답데이트</b>코스로 좋은듯합니다! 서울여행와서 묵은 청량리호텔인 더 디자이너스... ',
   'bloggername': '내 속엔 내가 너무도 많아',
   'bloggerlink': 'https://blog.naver.com/clf123',
   'postdate': '20211004'},
  {'title': '서울, 매화 여행은 <b>용답</b>역 청계천 매화거리로',
   'link': 'https://blog.naver.com/yo_onji?Redirect=Log&logNo=222686137521',
   'description': '걷다보면 보이는 홍매화 ♥️ 대나무와 함께 피어있는데 색이 참 예뻤어요 ! 대나무도 푸릇푸릇 넘 예뻤구용 쉴수 있게 벤치도 있었어용! <b>데이트</b>나 산책으로 <b>용답</b>역 매화거리, 청계천 매화거리 추천드려요',
   'bloggername': 'MY world',
   'bloggerlink': 'https://blog.naver.com/yo_onji',
   'postdate': '20220329'},
  {'title': '서울 매화 명소 : <b>용답</b>역 청계천 하동매실거리 (ft. 개화 정도... ',
   'link': 'https://blog.naver.com/bluesspace?Redirect=Log&logNo=222678216282',
   'description': '서울 매화 명소: <b>용답</b>역 청계천 하동매실거리 (ft. 개화 정도, 위치) 안녕하세요! 얼마 전 뉴스에서...